# Timer

In [4]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 9, 17, 30

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [5]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '30m'
higher_timeframe = '6h'
opt_limit = 300
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [15, 20, 25, 30, 35]},
              'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                        'slowd_period': [3], 'low_bound': [10, 15, 20, 25, 30]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 30m/6h, trade type is sell
Number of combinations is 75


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [04:05<00:00,  3.27s/it]


# Check local statistics

In [2]:
# 15m/4h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('forecast_rank', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
24,STOCH_RSI,12,35,9,7,3,30,88.16,81.58,90.79,92.11,93.42,93.42,90.79,93.42,92.11,89.47,90.79,93.42,90.79,93.42,93.42,94.74,90.79,92.11,88.16,90.79,90.79,88.16,90.79,93.42,-0.29,-0.44,-1.16,-1.55,-1.66,-2.15,-2.64,-2.18,-2.12,-2.05,-2.01,-2.03,-2.12,-2.12,-2.15,-2.26,-2.54,-2.30,-2.02,-1.88,-1.97,-2.05,-2.63,-3.00,76,91.119167,-1.971667,1605.056667,-149.846667
23,STOCH_RSI,12,35,9,7,3,25,86.96,81.16,91.30,92.75,94.20,94.20,91.30,92.75,92.75,88.41,89.86,92.75,89.86,92.75,92.75,94.20,89.86,91.30,86.96,89.86,89.86,86.96,89.86,92.75,-0.29,-0.45,-1.15,-1.55,-1.63,-2.17,-2.64,-2.20,-2.37,-2.17,-2.23,-2.13,-2.19,-2.13,-2.13,-2.24,-2.44,-2.26,-1.97,-1.92,-1.87,-2.04,-2.61,-3.02,69,90.640000,-1.991667,1424.160000,-137.425000
49,STOCH_RSI,14,35,9,7,3,30,88.14,79.66,91.53,94.92,94.92,96.61,93.22,94.92,94.92,91.53,93.22,94.92,93.22,94.92,94.92,96.61,93.22,93.22,89.83,89.83,89.83,88.14,91.53,94.92,-0.29,-0.41,-1.15,-1.54,-1.69,-2.26,-2.81,-2.33,-2.60,-2.34,-2.24,-2.30,-2.22,-2.13,-2.12,-2.20,-2.63,-2.26,-2.06,-2.05,-2.02,-2.04,-2.48,-3.02,59,92.445833,-2.049583,1324.304167,-120.925417
48,STOCH_RSI,14,35,9,7,3,25,87.04,77.78,90.74,94.44,94.44,96.30,92.59,94.44,94.44,90.74,92.59,94.44,92.59,94.44,94.44,96.30,92.59,92.59,88.89,88.89,88.89,87.04,90.74,94.44,-0.29,-0.40,-1.12,-1.53,-1.59,-2.49,-2.83,-2.36,-2.62,-2.36,-2.27,-2.31,-2.23,-2.14,-2.11,-2.20,-2.54,-2.20,-2.02,-1.98,-1.88,-1.94,-2.18,-3.00,54,91.742500,-2.024583,1174.095000,-109.327500
74,STOCH_RSI,16,35,9,7,3,30,90.91,80.30,84.85,86.36,86.36,87.88,84.85,86.36,86.36,83.33,84.85,84.85,81.82,83.33,83.33,84.85,81.82,86.36,83.33,83.33,81.82,83.33,84.85,87.88,-0.29,-0.47,-1.08,-1.38,-1.46,-2.09,-2.38,-2.03,-2.05,-2.02,-1.80,-1.70,-1.64,-1.85,-1.99,-2.09,-2.37,-2.00,-2.08,-1.74,-2.16,-2.02,-1.99,-2.50,66,84.721250,-1.799167,971.602500,-118.745000
22,STOCH_RSI,12,35,9,7,3,20,86.76,79.41,83.82,85.29,83.82,83.82,82.35,83.82,82.35,77.94,80.88,82.35,77.94,80.88,82.35,82.35,77.94,83.82,77.94,83.82,83.82,79.41,80.88,82.35,-0.29,-0.51,-0.96,-1.30,-1.32,-1.75,-1.91,-1.88,-1.83,-1.73,-1.59,-1.58,-1.58,-1.73,-1.69,-1.94,-1.88,-1.64,-1.45,-1.55,-1.66,-1.79,-1.99,-2.80,68,81.921250,-1.597917,810.645000,-108.658333
73,STOCH_RSI,16,35,9,7,3,25,90.00,78.33,83.33,85.00,85.00,86.67,83.33,85.00,85.00,81.67,83.33,83.33,80.00,81.67,81.67,83.33,80.00,85.00,81.67,81.67,81.67,81.67,83.33,86.67,-0.29,-0.44,-1.06,-1.38,-1.39,-2.01,-2.24,-2.03,-2.02,-1.99,-1.79,-1.70,-1.58,-1.82,-1.69,-1.99,-2.20,-1.81,-1.63,-1.60,-1.87,-1.88,-1.93,-2.42,60,83.264167,-1.698333,795.850000,-101.900000
47,STOCH_RSI,14,35,9,7,3,20,88.89,77.78,83.33,85.19,83.33,85.19,83.33,85.19,85.19,81.48,83.33,83.33,79.63,81.48,81.48,83.33,79.63,85.19,81.48,81.48,81.48,79.63,81.48,83.33,-0.28,-0.42,-1.03,-1.34,-1.38,-1.90,-2.02,-1.99,-2.02,-1.99,-1.80,-1.79,-1.62,-1.85,-1.82,-1.98,-2.19,-1.77,-1.63,-1.60,-1.79,-1.85,-1.87,-2.56,54,82.715833

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            

# Save new config data to config file

In [7]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [35]},
                  'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                            'slowd_period': [3], 'low_bound': [25]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)